In [15]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [ ]:
%%html
<script>
    $([IPython.events]).on("kernel_ready.Kernel", function () {
        $('div#header-container').hide();
        $('div#maintoolbar').hide();
    });
</script>

In [ ]:
%%html
<script>
    code_show=true; 
    function code_toggle() {
        if (code_show){
            $('div.input').hide();
        } else {
            $('div.input').show();
        }
        code_show = !code_show
    } 
    $( document ).ready(code_toggle);
</script>

<img src="./qarnot_ligne.png" 
     width="30%" 
     align=right
     alt="Dask logo">
     

# JupyterLab on Qarnot

## Add your Qarnot token

In [ ]:
import os
import io
import pandas as pd
import ipywidgets as widgets
from tkinter import Tk, filedialog
from IPython.display import clear_output, display, HTML

In [ ]:
token = widgets.Password(
    placeholder='Enter token',
    description='Qarnot token:',
    disabled=False,
    layout=widgets.Layout(width='40%')
)
display(token)

## Add your public SSH key

In [ ]:
ssh_key = widgets.Text(
    placeholder='Enter Key',
    description='SSH Public Key:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)
display(ssh_key)

## Task and bucket names

In [ ]:
from ipywidgets import Layout, Box, Label, Text

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

form_items = [
    Box([Label(value='Task name:'),
        Text(value='jupyterlab', placeholder='Enter task name')], layout=form_item_layout),
    Box([Label(value='Input bucket name:'),
        Text(value='jupyterlab-in', placeholder='Enter input bucket name')], layout=form_item_layout),
    Box([Label(value='Output bucket name:'),
        Text(value='jupyterlab-out', placeholder='Enter input bucket name')], layout=form_item_layout),
]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='50%'
))

form

## Select the software you want

Select one of the available JupyterLab images that come pre-installed with popular Machine Learning softwares

In [ ]:
soft_options = [
    ('Base','qarnotlab/jupyterlab-base'), ('Scikit-learn', 'qarnotlab/jupyterlab-sklearn'), 
    ('Tensorflow', 'qarnotlab/jupyterlab-tensorflow-cpu'), ('Pytorch', 'qarnotlab/jupyterlab-pytorch-cpu')
]

In [ ]:
soft = widgets.Select(
    options=soft_options,
    value='qarnotlab/jupyterlab-base',
    rows=4,
    description='JupyterLab Software:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)
display(soft)

# Upload your data to binder 

In [ ]:
file = widgets.FileUpload(
    multiple=True  # True to accept multiple files upload else False
)
display(file)

def on_upload_change(change):
    # create input folder
    ! mkdir -p input_binder/
    
    for i, df in enumerate(file.value):
        content = file.value[df]['content']
        with open('input_binder/'+df, 'wb') as f: f.write(content)

file.observe(on_upload_change, names='_counter')

## Use previous output bucket as input

In [ ]:
check = widgets.Checkbox(
    value=False,
    description='Check to use previous output bucket as input',
    disabled=False,
    indent=False
)

display(check)

## Launch JupyterLab on Qarnot

In [ ]:
import os
from run_qarnot import submit_task
from ipywidgets import Button, Output, Layout
from traitlets import traitlets

class LoadedButton(widgets.Button):
    """A button that can holds a value as a attribute."""

    def __init__(self, value=None, *args, **kwargs):
        super(LoadedButton, self).__init__(*args, **kwargs)
        # Create the value attribute.
        self.add_traits(value=traitlets.Any(value))

button = LoadedButton(description="Start JupyterLab on Qarnot", layout=Layout(width='auto'))
output = Output()
display(button, output)

def on_button_clicked(b):
    
    # Dictionary for data storage
    param_dict = {'token':'', 'ssh_key':'', 'task':'', 'in_bucket':'', 'out_bucket':'',
                  'docker_repo':'', 'use_output_bucket':''}
    param_list = list(param_dict)
    
    # Retrieve data from form in dictionary
    output.clear_output()
    with output:
        try:
            param_dict['token'] = token.value
            param_dict['ssh_key'] = ssh_key.value
            # Retrieve data from form in dictionary
            for _, (key, elem) in enumerate(zip(param_list[2:5], form.children)):
                param_dict[key] = str(elem.children[1].value)
            param_dict['soft'] = soft.value
            param_dict['use_output_bucket'] = check.value
        except IndexError:
            print("Some fields were not properly filled")
        if param_dict['ssh_key'] == '' or param_dict['token'] == '':
            return(print('Some fields were not properly filled'))
    
    # Launch computation
        ! mkdir -p logs/
        # Launch task and get port, link and uuid
        forward_port, link, uuid = submit_task(param_dict)
        b.value = uuid
        ssh_cmd = "ssh -L 8888:localhost:8888 -o StrictHostKeyChecking=no root@forward01.qarnot.net -p "+str(forward_port)
        
        # Command to establish ssh connection
        ssh_prompt = widgets.HTML(
            value=f"<b style=font-size:12px;'>{'Copy and paste the following command in your terminal to connect via SSH with Qarnot'}</b>"
        )
        
        # Generate link to JupyterLab instance (with secret token)
        html_link = "<a href="+link+" target='_blank'>JupyterLab</a>"
        text = widgets.HTML(
            value = f"<b style=font-size:12px;'>{'Click on the link below to get access to your JupyterLab instance:'}</b>"
        )
        hyper_link = widgets.HTML(value=html_link,)
        
        # Display
        display(ssh_prompt)
        print(ssh_cmd)
        display(text)
        display(hyper_link)
        
button.on_click(on_button_clicked)

## Get results and abort task

In [ ]:
from run_qarnot import abort_task

button_abort = Button(description="Abort Task", layout=Layout(width='auto'))
output_abort = Output()
display(button_abort, output_abort)

def on_button_clicked_abort(b):
    
    # Launch computation
    output_abort.clear_output()
    with output_abort:
        print('Aborting Task...')
        abort_task(token.value, button.value)

button_abort.on_click(on_button_clicked_abort)

## Download results locally

In [ ]:
from shutil import make_archive
from IPython.display import FileLink

download_button = Button(description="Download outputs", layout=Layout(width='auto'))
download_output = Output()
display(download_button, download_output)

def on_download_button_clicked(b):
    download_output.clear_output()
    with download_output:
        try:
            print('Compressing outputs into .zip file...')
            make_archive('output', 'zip', 'outputs_binder/', verbose = 10)
            link = FileLink(
                path='output.zip', 
                result_html_prefix='Your output .zip file is ready ! \
                                    Click the following link to download it: '
            )
            display(link)
            
        except FileNotFoundError:
            print("Output files not available")
        
download_button.on_click(on_download_button_clicked)